In [51]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [52]:
data = pd.read_csv('./sonar_csv.csv')
dataset = data.values

In [53]:
#Particion
X = dataset[:,0:60].astype('float64')
y = dataset[:,60]

In [54]:
#Codificar la variable de salida, con one-hot-encoder
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [55]:
y

array(['Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Mine', 'Mine', 'Mine', 'Mine', 'Mine', 'Mine', 'Mine',
       'Mine', 'Mine', 'Mine', 'Mine', 'Mine', 'Mine', 'Mine', 'Mine',
      

 **Especificacion**
 1. Una capa oculta completamente conectada
 2. Funcion de activacion ReLu
 3. Capa salida una sola neurona funcion de activacion sigmoide
 4. Funcion de perdida binary_crossentropy
 5. Optimizer adam y Accuracy como metrica

In [56]:
def create_model():
    model = Sequential()
    model.add(Dense(60, input_dim = 60, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [57]:
estimador = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 5, verbose = 0)

In [58]:
kfold = StratifiedKFold(n_splits = 10, shuffle=True)
results = cross_val_score(estimador, X, encoded_y, cv = kfold)
print(" Linea base: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 Linea base: 82.71% (9.56%)


# Optimizar el rendimiento con procesamiento de datos

In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [60]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs = 100, batch_size=5, verbose = 0)))

In [61]:
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)

In [62]:
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 86.57% (7.30%)


## Topologia pequeña

In [64]:
def create_smaller():
    model = Sequential()
    model.add(Dense(30, input_dim = 60, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [65]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs = 100, batch_size=5, verbose = 0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 85.14% (8.87%)


## Topologia grande

In [66]:
def create_larger():
    model = Sequential()
    model.add(Dense(60, input_dim = 60, activation = 'relu'))
    model.add(Dense(30, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [67]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs = 100, batch_size=5, verbose = 0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 86.05% (8.22%)
